In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini
from google import genai
client = genai.Client()
MODEL_ID = "gemini-1.5-flash"

In [ ]:
# Importações para checkboxes de notícias
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML
from datetime import date

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import json

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, noticas_procuradas):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente buscador #################################################
        instruction="""
          Você é um agente buscador de notícias no Google. Sua tarefa é usar a ferramenta de busca do Google (google_seacrh) para recuperar as últimas notícias relevantes sobre o tópico abaixo.
          Foque em no máximo 6 notícias relevantes (de portais confiáveis, como Valor Econômico, CNN Brasil, G1) das últimas 24 horas. As notícias devem ser atuais, de no máximo 24h antes do dia e hora de agora.
          Você deve retornar sua resposta ESTRUTURADA no seguinte formato JSON:
          [
            {
              "titulo": "Título da notícia",
              "portal": "Nome do portal",
              "url": "URL limpa e acessível diretamente no navegador, sem redirecionadores"
              "resumo": "Resumo breve da notícia - apenas 1 frase"
            },
            ...
          ]
        ⚠️ Importante:
        - Não use formatação Markdown.
        - Não use negrito, listas numeradas ou bullet points.
        - Retorne apenas o JSON puro.
        - Nunca use links de redirecionamento (como vertexaisearch.cloud.google.com). Sempre prefira o link direto do site da notícia.
        """,
        description="Agente que busca notícias no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico:{topico}\nNotícias buscadas: {noticas_procuradas}"

    # Executa o agente
    noticias_buscadas = call_agent(buscador, entrada_do_agente_buscador)
    return noticias_buscadas

In [ ]:
######################################
# --- Agente 2: Resumidor de notícia --- #
######################################
def agente_resumidor(topico, resumos_feitos):
    resumidor = Agent(
        name="agente_resumidor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um assistente que resume cada notícia em um parágrafo fluido e coerente entre 5 a 7 linhas.
            Destaque os principais fatos, nomes e datas, com linguagem empresarial e foque apenas no conteúdo da notícia selecionada, sem dar opiniões.
            A saída deve ser:
            Título da notícia, em negrito;
            Link da notícia;
            Resumo da notícia.
          """,
        description="Agente resumidor de notícias"
    )
    entrada_do_agente_resumidor = f"Tópico: {topico}\nResumo: {resumos_feitos}"
    # Executa o agente
    resumo = call_agent(resumidor, entrada_do_agente_resumidor)
    return resumo

In [ ]:
# Função utilitária para formatar as notícias em markdown
def to_markdown(noticias):
    if isinstance(noticias, str):
        return Markdown(noticias)
    elif isinstance(noticias, list):
        texto = "\n\n".join(
            f"**{n.get('titulo', 'Sem título')}**\n{n.get('url', 'Sem URL')}" for n in noticias
        )
        return Markdown(texto)
    else:
        return Markdown("Formato de notícia não reconhecido.")

In [ ]:
# Função para exibir checkboxes e capturar os selecionados
def selecionar_noticias(noticias):
    checkboxes = []
    linhas_widgets = []

    for noticia in noticias:
        titulo = noticia.get('titulo', 'Sem título')
        portal = noticia.get('portal', 'Fonte desconhecida')
        resumo = noticia.get('resumo', 'Sem resumo.')

        # Checkbox com descrição simples
        cb = widgets.Checkbox(
            value=False,
            description='',
            indent=False,
            layout=widgets.Layout(width='20px')
        )
        cb.noticia = noticia
        checkboxes.append(cb)

        # Widget de título + portal
        titulo_html = widgets.HTML(
            f"<b>{titulo}</b> <span style='color:gray;'>({portal})</span>"
        )

        # Widget de resumo indentado
        resumo_html = widgets.HTML(
            f"<div style='margin-left:6px; color:#444; font-style:italic;'>{resumo}</div>"
        )

        # Agrupa visualmente os elementos
        linha = widgets.HBox([cb, widgets.VBox([titulo_html, resumo_html])], layout=widgets.Layout(align_items='flex-start', gap='6px'))
        linhas_widgets.append(linha)

    # Botão para confirmar
    botao = widgets.Button(description="Confirmar seleção")
    output = widgets.Output()

    def on_click(_):
        selecionadas = [cb.noticia for cb in checkboxes if cb.value]
        with output:
            output.clear_output()
            if selecionadas:
                print("✅ Notícias selecionadas. Gerando resumo...")
                #resumo = agente_resumidor(topico, selecionadas)
                # Transforma a lista em string formatada para o agente
                noticias_em_texto = ""
                for n in selecionadas:
                    noticias_em_texto += f"Título: {n.get('titulo')}\nFonte: {n.get('portal')}\nURL: {n.get('url')}\n\n"
                resumo = agente_resumidor(topico, noticias_em_texto)
                display(Markdown("### 📝 RESUMO DAS NOTÍCIAS SELECIONADAS"))
                display(to_markdown(resumo))
            else:
                print("⚠️ Nenhuma notícia selecionada.")

    botao.on_click(on_click)

    print("🗞️ Marque abaixo as notícias mais relevantes para serem resumidas:\n")
    display(widgets.VBox(linhas_widgets))
    display(botao, output)

In [ ]:
# Execução da pipeline
data_de_hoje = date.today().strftime('%d/%m/%Y %H:%M')

print("🚀 Iniciando o Sistema de Resumo de Notícias para Radar com 2 Agentes 🚀")

topico = input("❓ Por favor, digite o TEMA sobre o qual você quer criar o resumo de notícias: ")

if not topico:
    print("Digite um tema")
else:
    print(f"🔎 Buscando notícias sobre: **{topico}**")
    noticias = agente_buscador(topico, data_de_hoje)
    if isinstance(noticias, str):
      # Remove bloco de código Markdown, se houver
      noticias = noticias.strip().strip("```json").strip("```").strip()
      noticias = json.loads(noticias)

    print("\n--- SELECIONE AS NOTÍCIAS RELEVANTES ---\n")
    selecionar_noticias(noticias)

🚀 Iniciando o Sistema de Resumo de Notícias para Radar com 2 Agentes 🚀
❓ Por favor, digite o TEMA sobre o qual você quer criar o resumo de notícias: gripe aviária
🔎 Buscando notícias sobre: **gripe aviária**

--- SELECIONE AS NOTÍCIAS RELEVANTES ---

🗞️ Marque abaixo as notícias mais relevantes para serem resumidas:



Button(description='Confirmar seleção', style=ButtonStyle())

Output()